# Generating a Mosaicked Image of Lake Mead

## Outline of steps for analysis

+ Identifying search parameters
    + AOI, time-window
    + Endpoint, Provider, catalog identifier ("short name")
+ Obtaining search results
    + Instrospect, examine to identify features, bands of interest
    + Wrap results into a DataFrame for easier exploration
+ Exploring & refining search results
    + Identify granules of highest value
    + Filter extraneous granules with minimal contribution
    + Assemble relevant filtered granules into DataFrame
    + Identify kind of output to generate
+ Data-wrangling to produce relevant output
    + Download relevant granules into Xarray DataArray, stacked appropriately
    + Do intermediate computations as necessary
    + Assemble relevant data slices into visualization

---

### Preliminary imports

In [ ]:
from warnings import filterwarnings
filterwarnings('ignore')
# data wrangling imports
import numpy as np
import pandas as pd
import xarray as xr
import rioxarray as rio
import rasterio

In [ ]:
# Imports for plotting
import hvplot.pandas
import hvplot.xarray
import geoviews as gv
from geoviews import opts
gv.extension('bokeh')

In [ ]:
# STAC imports to retrieve cloud data
from pystac_client import Client
from osgeo import gdal
# GDAL setup for accessing cloud data
gdal.SetConfigOption('GDAL_HTTP_COOKIEFILE','~/.cookies.txt')
gdal.SetConfigOption('GDAL_HTTP_COOKIEJAR', '~/.cookies.txt')
gdal.SetConfigOption('GDAL_DISABLE_READDIR_ON_OPEN','EMPTY_DIR')
gdal.SetConfigOption('CPL_VSIL_CURL_ALLOWED_EXTENSIONS','TIF, TIFF')

### Convenient utilities

These functions could be placed in module files for more developed research projects. For learning purposes, they are embedded within this notebook.

In [ ]:
# simple utility to make a rectangle with given center of width dx & height dy
def make_bbox(pt,dx,dy):
    '''Returns bounding-box represented as tuple (x_lo, y_lo, x_hi, y_hi)
    given inputs pt=(x, y), width & height dx & dy respectively,
    where x_lo = x-dx/2, x_hi=x+dx/2, y_lo = y-dy/2, y_hi = y+dy/2.
    '''
    return tuple(coord+sgn*delta for sgn in (-1,+1) for coord,delta in zip(pt, (dx/2,dy/2)))

In [ ]:
# simple utility to plot an AOI or bounding-box
def plot_bbox(bbox):
    '''Given bounding-box, returns GeoViews plot of Rectangle & Point at center
    + bbox: bounding-box specified as (lon_min, lat_min, lon_max, lat_max)
    Assume longitude-latitude coordinates.
    '''
    # These plot options are fixed but can be over-ridden
    point_opts = opts.Points(size=12, alpha=0.25, color='blue')
    rect_opts = opts.Rectangles(line_width=0, alpha=0.1, color='red')
    lon_lat = (0.5*sum(bbox[::2]), 0.5*sum(bbox[1::2]))
    return (gv.Points([lon_lat]) * gv.Rectangles([bbox])).opts(point_opts, rect_opts)

In [ ]:
# utility to extract search results into a Pandas DataFrame
def search_to_dataframe(search):
    '''Constructs Pandas DataFrame from PySTAC Earthdata search results.
    DataFrame columns are determined from search item properties and assets.
    'asset': string identifying an Asset type associated with a granule
    'href': data URL for file associated with the Asset in a given row.'''
    granules = list(search.items())
    assert granules, "Error: empty list of search results"
    props = list({prop for g in granules for prop in g.properties.keys()})
    tile_ids = map(lambda granule: granule.id.split('_')[3], granules)
    rows = (([g.properties.get(k, None) for k in props] + [a, g.assets[a].href, t])
                for g, t in zip(granules,tile_ids) for a in g.assets )
    df = pd.concat(map(lambda x: pd.DataFrame(x, index=props+['asset','href', 'tile_id']).T, rows),
                   axis=0, ignore_index=True)
    assert len(df), "Empty DataFrame"
    return df

---

## Identifying search parameters

[Lake Mead](https://en.wikipedia.org/wiki/Lake_Mead) is a water reservoir in southwestern United States and is significant for irrigation in neighboring states. The lake has experienced significant drought over the past decade and particularly between 2020 & 2023.

In [ ]:
lake_mead = (-114.754, 36.131)
AOI = make_bbox(lake_mead, 0.1, 0.1)
DATE_RANGE = "2023-03-01/2023-04-15"

In [ ]:
# Optionally plot the AOI
basemap = gv.tile_sources.OSM(width=500, height=500, padding=0.1)
plot_bbox(AOI) * basemap

In [ ]:
search_params = dict(bbox=AOI, datetime=DATE_RANGE)
print(search_params)

---

## Obtaining search results

As usual, we'll specify the search endpoint, provider, & catalog. For the DSWx data products these are as follows.

In [ ]:
ENDPOINT = 'https://cmr.earthdata.nasa.gov/stac'
PROVIDER = 'POCLOUD'
COLLECTIONS = ["OPERA_L3_DSWX-HLS_V1_1.0"]
# Update the dictionary opts with list of collections to search
search_params.update(collections=COLLECTIONS)
print(search_params)

In [ ]:
catalog = Client.open(f'{ENDPOINT}/{PROVIDER}/')
search_results = catalog.search(**search_params)

We convert the search results to a `DataFrame` for easier perusal.

In [ ]:
df = search_to_dataframe(search_results)
display(df.head())
df.info()

Clean DataFrame `df` in ways that make sense (e.g., dropping unneeded columns/rows, casting columns as fixed datatypes, setting the index, etc.).

In [ ]:
df.datetime = pd.DatetimeIndex(df.datetime)
df = df.drop(['start_datetime', 'end_datetime'], axis=1)
df = df.rename({'eo:cloud_cover':'cloud_cover'}, axis=1)
df['cloud_cover'] = df['cloud_cover'].astype(np.float16)
for col in ['asset', 'href', 'tile_id']:
    df[col] = df[col].astype(pd.StringDtype())
df = df.set_index('datetime').sort_index()

In [ ]:
display(df.head())
df.info()

---

## Exploring & refining search results

We can look at the different assets and filter for the `B01_WTR` band.

In [ ]:
df.asset.value_counts()

We can also see how much cloud cover there is in our search results.

In [ ]:
df.cloud_cover.agg(['min','mean','median','max'])

We can filter for both of these from the `DataFrame` using boolean `Series`.

In [ ]:
c1 = (df.cloud_cover <= 10)
c2 = (df.asset.str.contains('B01_WTR'))
b01_wtr = df.loc[c1 & c2].drop(['asset', 'cloud_cover'], axis=1)
b01_wtr

Finally, we can see how many geographic different tiles intersect our AOI.

In [ ]:
b01_wtr.tile_id.value_counts()

---

## Data-wrangling to produce relevant output

This time, we'll use a technique called *mosaicking* to combine raster data from adjacent tiles into a single raster data set. This requires the `merge` function as before but also the `array_bounds` function from `rasterio.transform` to get the coordinates aligned properly.

In [ ]:
from rasterio.merge import merge
from rasterio.transform import array_bounds

We've used the function `merge` before with distinct raster data sets corresponding to a single tile. This time, the raster data merged will come from adjacent tiles.

In [ ]:
%%time
mosaicked_img, mosaic_transform = merge(b01_wtr.href)

The output again is consists of a NumPy array and coordinate transformation. These can be combined using the `array_bounds` function to determine the UTM coordinates of the corners of the resulting mosaicked image.

In [ ]:
print(f"{type(mosaicked_img)=}")
print(mosaicked_img.shape)
print(f"{type(mosaic_transform)=}")
print(mosaic_transform)

In [ ]:
bounds = array_bounds(*mosaicked_img.shape[1:], mosaic_transform)

bounds

We'll bundle these commands within a function that accepts a DataFrame of search results as input and returns an Xarray `DataArray` containing the mosaicked raster image combining all the input raster images.

In [ ]:
def mosaic_and_dataset(granule_dataframe):
    '''
    This method takes a pandas dataframe from a PySTAC query and mosaics the tiles it points to.  
    The raster is then loaded into an xarray data array and returned
    '''

    mosaicked_img, mosaic_transform = merge(granule_dataframe.href)
    bounds = array_bounds(*mosaicked_img.shape[1:], mosaic_transform)
    
    with rasterio.open(granule_dataframe.iloc[0].href) as ds:
        # extract CRS string
        crs = str(ds.crs).split(':')[-1]

    # the x and y resolution of the image is available in image metadata
    x_res = np.abs(mosaic_transform[0])
    y_res = np.abs(mosaic_transform[4])

    xmin, ymin, xmax, ymax = bounds

    lon = np.arange(xmin, xmax, x_res)
    lat = np.arange(ymax, ymin, -y_res)

    da = xr.DataArray(
        data=mosaicked_img,
        dims=["band", "lat", "lon"],
        coords=dict(
            lon=(["lon"], lon),
            lat=(["lat"], lat),
        ),
        attrs=dict(
            description="OPERA DSWx B01",
            units=None,
        ),
    )
    da.rio.write_crs(crs, inplace=True)

    return da
    

In [ ]:
%time dataset = mosaic_and_dataset(b01_wtr)

In [ ]:
dataset

Once combined into a dataset, the mosaicked DataArray can be plotted as we've seen previously.

In [ ]:
# Define a colormap
COLORS = [ (255, 255, 255, 0.1) for _ in range(256) ] # setting all colors to transparent initially
COLORS[0] = (0, 255, 0, 0.1) # Setting not water class to green
COLORS[1] = (0, 0, 255, 1) # Open surface water
COLORS[2] = (0, 0, 255, 1) # Partial surface water

In [ ]:
image_opts = dict(
                    x='lon',
                    y='lat', 
                    project=True, rasterize=True,
                    framewise=False, 
                    cmap=COLORS, 
                    colorbar=False,
                    tiles = gv.tile_sources.ESRI,
                    frame_width=1000, frame_height=1000,
                 )
layout_opts = dict(
                    title = 'Lake Mead, NV USA - mosaicked water extent',
                    xlabel='Longitude (degrees)',
                    ylabel='Latitude (degrees)',
                    fontscale=1.25,
                   )

We can use the builtin Python `slice` function to extract downsampled images quickly before trying to view the entire image.

In [ ]:
%%time
N = 100
view = dataset.isel(lon=slice(0,None,N), lat=slice(0,None,N))
img = view.hvplot.image(**image_opts).opts(**layout_opts)

img

---